In [ ]:
import argparse
from corner import quantile
import json
import pickle
import sys
import numpy as np
from tools import *

In [ ]:
parser = argparse.ArgumentParser(description="Produces the read_results_updated.data with the correct values relative to the MCMC run")
parser.add_argument('SETTING_FILE',type=str,help="setting file to consider")
parser.add_argument("-c", "--cut_mcmc", type=int, dest="cut_mcmc", default=0,
                    help="cut the first <c> steps of the mcmc to ignore them")
args = parser.parse_args()
settings = args.SETTING_FILE
cut_mcmc = args.cut_mcmc

In [ ]:
present_program(sys.argv[0])

setting_name = settings.replace(".py","") 
backup_path  = "backup_results"

savemcmc_path="./"+backup_path+"/"+setting_name.replace("settings_","mcmc_")+"/"
savefig_path="./"+backup_path+"/"+setting_name.replace("settings_","")+"/" 

mcmc_file=savemcmc_path+setting_name.replace("settings","mcmc_smpl")+".json"
param_file=savemcmc_path+setting_name.replace("settings","mcmc_prm")+".dat"

#MCMC sample
with open(mcmc_file, 'r') as mcmc_file:
    samples_mcmc = np.array(json.load(mcmc_file))[cut_mcmc:]

#parameters' name
param_mcmc=[]
with open(param_file,"r") as param_file:
    param_mcmc=(param_file.readlines())
    
for i in range(len(param_mcmc)):
    param_mcmc[i]=param_mcmc[i].replace(",\n","")

kwargs_results_updated={} 
n_ra=0
n_dec=0  

for i in range(len(param_mcmc)):
    val = float(quantile(samples_mcmc[:,i],q=.5))
    if param_mcmc[i]!="ra_image" and param_mcmc[i]!="dec_image":
        kwargs_results_updated[param_mcmc[i]]=val 
    elif param_mcmc[i]=="ra_image":
        kwargs_results_updated["ra_image_"+str(n_ra)]=val          
        n_ra+=1
    else:
        kwargs_results_updated["dec_image_"+str(n_dec)]=val          
        n_dec+=1

In [ ]:
dataset = kwargs_results_updated
outputFile = savefig_path+'/read_results_updated.data'

with open(outputFile, 'wb') as file:
    pickle.dump(dataset, file)

In [ ]:
success(sys.argv[0])